In [70]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_palette('Set2')

In [71]:
def oracle(numbers, goal, wires, aux_wires):
    """
    Oracle:
      marks states where the sum of the selected numbers is equal to the goal.
      args:
        numbers: list of numbers to sum
        goal: the goal number
        wires: the wires to apply the oracle to
        aux_wires: the auxiliary wires to perform the computation, mind for overflow
        """
    
    def add_k_fourier(k, wires):
        for j in range(len(wires)):
            qml.RZ(k * np.pi / (2**j), wires=wires[j])

    def compute_sum_in_fourier_basis():
        qml.QFT(wires=aux_wires)
        for idx, wire in enumerate(wires):
            qml.ctrl(add_k_fourier, control=wire)(numbers[idx], wires=aux_wires)
        qml.adjoint(qml.QFT)(wires=aux_wires)
        
    compute_sum_in_fourier_basis()

    qml.FlipSign(goal, aux_wires) ## check
    
    qml.adjoint(compute_sum_in_fourier_basis)()

def grover_circuit(numbers, goal, rep):
    num_wires = len(numbers)
    aux_wires = [num_wires + i for i in range(num_wires)]
    dev = qml.device("default.qubit", wires=num_wires + num_wires, shots=1)
    
    @qml.qnode(dev)
    def circuit():
        qml.Snapshot("init", measurement= qml.probs(wires=range(num_wires)))
        # Step 1: Apply Hadamard gates to all variable wires
        for wire in range(num_wires):
            qml.Hadamard(wires=wire)
        qml.Snapshot("init", measurement= qml.probs(wires=range(num_wires)))

        for i in range(rep):
            # Step 2: Apply the oracle
            oracle(numbers, goal, range(num_wires), aux_wires)
            #qml.Snapshot(f"oracle_{i + 1}", measurement= qml.probs(wires=range(num_wires)))
            
            # Step 3: Apply Grover's diffusion operator
            qml.GroverOperator(wires=range(num_wires))
            qml.Snapshot(f"diffusion_{i + 1}", measurement= qml.probs(wires=range(num_wires)))
        
        return qml.sample(wires=range(num_wires))
    
    return circuit


In [72]:
numbers = [15,3,5,8] 
rep = 2 # number of grover iterations
df = pd.DataFrame()
global_goal = 14
for goal in range(global_goal + 1):
    circuit = grover_circuit(numbers, goal, rep)
    result = circuit()
    df[goal] = result


In [73]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1
1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1
2,0,0,0,0,1,1,1,0,1,0,0,0,1,1,1
3,0,0,0,0,0,0,1,1,0,1,1,1,1,1,0


In [74]:
validity_row = {i: x.item() for 
 (i,x) in 
 zip(list(range(15)),
      np.dot(df.to_numpy().T, np.array(numbers)))}
validity_row

{0: 0,
 1: 15,
 2: 18,
 3: 3,
 4: 20,
 5: 5,
 6: 13,
 7: 23,
 8: 8,
 9: 11,
 10: 26,
 11: 11,
 12: 28,
 13: 13,
 14: 23}

In [75]:
df.loc[len(df)]= validity_row

In [76]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1
1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1
2,0,0,0,0,1,1,1,0,1,0,0,0,1,1,1
3,0,0,0,0,0,0,1,1,0,1,1,1,1,1,0
4,0,15,18,3,20,5,13,23,8,11,26,11,28,13,23
